In [4]:
# colab환경용
# !apt-get update

# !apt-get install g++ openjdk-8-jdk python-dev python3-dev

# !pip3 install JPype1-py3

# !pip3 install konlpy

# !JAVA_HOME = /Library/Java/JavaVirtualMachines/zulu-15.jdk/Contents/Home

In [5]:
# 패키지 임포트
import re 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from konlpy.tag import Okt

In [6]:
# 데이터 불러오기 
df = pd.read_csv('./dataset/홍대_맛집_리뷰_데이터_ver5.csv')

In [7]:
df.head(10)

Unnamed: 0 name  average_rating    user  user_rating  \
0           0  후계동             4.5     풀내음          5.0   
1           1  후계동             4.5      ㅇㅇ          5.0   
2           2  후계동             4.5      핑구          5.0   
3           3  후계동             4.5      :ᴅ          3.0   
4           4  후계동             4.5     Vin          5.0   
5           5  후계동             4.5     손민영          5.0   
6           6  후계동             4.5  장슐랭가이드          1.0   
7           7  후계동             4.5     이우진          5.0   
8           8  후계동             4.5     정찰병          4.0   
9           9  후계동             4.5     쿠집사          5.0   

                                             comment  \
0                                                NaN   
1                                         ❤️❤️❤️❤️❤️   
2                                           오래 해주세요🙏   
3  웨이팅해서 먹을 맛은 아니에요… 닭보쌈은 보통이었고 비빔국수라고 해서 당연히 국물 ...   
4       와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!   
5  오늘의 메뉴 너무 좋아요. 어쩜 갖가지 닭요리를 그렇게 잘하시죠?!! 이런 리뷰 잘...   
6                                  여기서 밥먹고 둘다 장염걸림;;   
7  최애 식당진짜 다 맛있고 가격이 엄청 착함오늘의 메뉴라고 매일 새로운 메뉴가 준비되...   
8  다음부터 사장님 김치볶음밥하지마세요. ㅠ ㅜ 라면만 하세요!하실꺼면 진짜 딴거말고 ...   
9                           이렇게 맛있어도 되나 할 정도로 맛있어요ㅎㅎ   

                                      url  target  
0  https://place.map.kakao.com/1183957472       1  
1  https://place.map.kakao.com/1183957472       1  
2  https://place.map.kakao.com/1183957472       1  
3  https://place.map.kakao.com/1183957472       0  
4  https://place.map.kakao.com/1183957472       1  
5  https://place.map.kakao.com/1183957472       1  
6  https://place.map.kakao.com/1183957472       0  
7  https://place.map.kakao.com/1183957472       1  
8  https://place.map.kakao.com/1183957472       1  
9  https://place.map.kakao.com/1183957472       1

In [8]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3130 entries, 0 to 3129
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      3130 non-null   int64  
 1   name            3130 non-null   object 
 2   average_rating  3130 non-null   float64
 3   user            3127 non-null   object 
 4   user_rating     3130 non-null   float64
 5   comment         2529 non-null   object 
 6   url             3130 non-null   object 
 7   target          3130 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 195.8+ KB


In [9]:
df.duplicated(['comment'])

0       False
1       False
2       False
3       False
4       False
        ...  
3125     True
3126     True
3127     True
3128     True
3129    False
Length: 3130, dtype: bool

In [10]:
df.drop_duplicates(['comment'])

Unnamed: 0   name  average_rating  user  user_rating  \
0              0    후계동             4.5   풀내음          5.0   
1              1    후계동             4.5    ㅇㅇ          5.0   
2              2    후계동             4.5    핑구          5.0   
3              3    후계동             4.5    :ᴅ          3.0   
4              4    후계동             4.5   Vin          5.0   
...          ...    ...             ...   ...          ...   
3120        3120  혼가츠홍대             2.9    조에          5.0   
3122        3122  혼가츠홍대             2.9  maya          1.0   
3123        3123  혼가츠홍대             2.9    어쩜          2.0   
3124        3124  혼가츠홍대             2.9   이정훈          5.0   
3129        3129  혼가츠홍대             2.9  myks          4.0   

                                                comment  \
0                                                   NaN   
1                                            ❤️❤️❤️❤️❤️   
2                                              오래 해주세요🙏   
3     웨이팅해서 먹을 맛은 아니에요… 닭보쌈은 보통이었고 비빔국수라고 해서 당연히 국물 ...   
4          와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!   
...                                                 ...   
3120  음식이 가격대비 양도 많고 맛있는 곳입니다!!!분위기랑 직원들의 반응이 중요하신 분...   
3122  여기 알바들이 설쳐대는게 아니라면 방송 탔었다고 이리 리뷰가 많은것인가요? 진심 평...   
3123        음. 웨이팅 십오분했는데 차게 식은 치즈가스가 나왔다. 왕혼가츠도 그냥 쏘쏘.   
3124      가격은 양에비해서 싸거나 적당하다고 봅니다. 기본 생돈까스는 정말 양이 많슴니다.   
3129                                           20160515   

                                         url  target  
0     https://place.map.kakao.com/1183957472       1  
1     https://place.map.kakao.com/1183957472       1  
2     https://place.map.kakao.com/1183957472       1  
3     https://place.map.kakao.com/1183957472       0  
4     https://place.map.kakao.com/1183957472       1  
...                                      ...     ...  
3120    https://place.map.kakao.com/22895023       1  
3122    https://place.map.kakao.com/22895023       0  
3123    https://place.map.kakao.com/22895023       0  
3124    https://place.map.kakao.com/22895023       1  
3129    https://place.map.kakao.com/22895023       1  

[2465 rows x 8 columns]

In [11]:
df['target'].value_counts().plot(kind = 'bar')

<AxesSubplot:>

## 결측치 제거

In [12]:
df.isnull().sum()

Unnamed: 0          0
name                0
average_rating      0
user                3
user_rating         0
comment           601
url                 0
target              0
dtype: int64

In [13]:
df = df.dropna()
df.isnull().sum()

Unnamed: 0        0
name              0
average_rating    0
user              0
user_rating       0
comment           0
url               0
target            0
dtype: int64

## 데이터 정제

In [14]:
df['comment'] = df['comment'].str.replace("[^ㄱ-하-ㅣ가-힣 ]", '')

df.head()

/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_37240/408186909.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment'] = df['comment'].str.replace("[^ㄱ-하-ㅣ가-힣 ]", '')


Unnamed: 0 name  average_rating user  user_rating  \
1           1  후계동             4.5   ㅇㅇ          5.0   
2           2  후계동             4.5   핑구          5.0   
3           3  후계동             4.5   :ᴅ          3.0   
4           4  후계동             4.5  Vin          5.0   
5           5  후계동             4.5  손민영          5.0   

                                             comment  \
1                                                      
2                                            오래 해주세요   
3  웨이팅해서 먹을 맛은 아니에요 닭보쌈은 보통이었고 비빔국수라고 해서 당연히 국물 없...   
4           와 여길 왜 이제알았죠 너무맛있고 반찬 하나하나 다 맛있네요 자주 갈게요   
5  오늘의 메뉴 너무 좋아요 어쩜 갖가지 닭요리를 그렇게 잘하시죠 이런 리뷰 잘 안 남...   

                                      url  target  
1  https://place.map.kakao.com/1183957472       1  
2  https://place.map.kakao.com/1183957472       1  
3  https://place.map.kakao.com/1183957472       0  
4  https://place.map.kakao.com/1183957472       1  
5  https://place.map.kakao.com/1183957472       1

In [15]:
df['comment'] = df['comment'].replace('', np.nan)
len(df)

2527

In [16]:
df.isnull().sum()

Unnamed: 0        0
name              0
average_rating    0
user              0
user_rating       0
comment           5
url               0
target            0
dtype: int64

In [17]:
 # 정규표현식으로 인해 공백이 된 리뷰 행 제거
df = df.dropna()
len(df)

2522

In [18]:
df.head()

Unnamed: 0 name  average_rating    user  user_rating  \
2           2  후계동             4.5      핑구          5.0   
3           3  후계동             4.5      :ᴅ          3.0   
4           4  후계동             4.5     Vin          5.0   
5           5  후계동             4.5     손민영          5.0   
6           6  후계동             4.5  장슐랭가이드          1.0   

                                             comment  \
2                                            오래 해주세요   
3  웨이팅해서 먹을 맛은 아니에요 닭보쌈은 보통이었고 비빔국수라고 해서 당연히 국물 없...   
4           와 여길 왜 이제알았죠 너무맛있고 반찬 하나하나 다 맛있네요 자주 갈게요   
5  오늘의 메뉴 너무 좋아요 어쩜 갖가지 닭요리를 그렇게 잘하시죠 이런 리뷰 잘 안 남...   
6                                    여기서 밥먹고 둘다 장염걸림   

                                      url  target  
2  https://place.map.kakao.com/1183957472       1  
3  https://place.map.kakao.com/1183957472       0  
4  https://place.map.kakao.com/1183957472       1  
5  https://place.map.kakao.com/1183957472       1  
6  https://place.map.kakao.com/1183957472       0

## 형태소 분석 및 백터화
---
* 단어장 생성

In [19]:
def get_morph(text):
    okt = Okt()
    pos = okt.pos(text)

    X = []
    for a1 in pos:
        X.append(f'{a1[0]}/{a1[1]}')
    return X

In [20]:
get_morph('안녕하세요 반갑습니다. 저는 김성겸입니다')

['안녕하세요/Adjective',
 '반갑습니다/Adjective',
 './Punctuation',
 '저/Noun',
 '는/Josa',
 '김성겸/Noun',
 '입니다/Adjective']

In [21]:
df = df.reset_index()

index_vectorizer = CountVectorizer(tokenizer=get_morph)
X = index_vectorizer.fit_transform(df['comment'].tolist())

In [22]:
index_vectorizer.vocabulary_

{'오래/Adverb': 7239,
 '해주세요/Verb': 10493,
 '웨이/Noun': 7572,
 '팅/Noun': 9854,
 '해서/Verb': 10461,
 '먹을/Verb': 3947,
 '맛/Noun': 3566,
 '은/Josa': 7647,
 '아니에요/Adjective': 6349,
 '닭/Noun': 2313,
 '보쌈/Noun': 4737,
 '보통/Noun': 4776,
 '이었고/Verb': 7813,
 '비빔국수/Noun': 5128,
 '라고/Josa': 3178,
 '당연히/Adjective': 2363,
 '국물/Noun': 915,
 '없을/Adjective': 7031,
 '줄/Noun': 8781,
 '알/Noun': 6647,
 '고/Josa': 714,
 '시켰는데/Verb': 5895,
 '이/Josa': 7692,
 '있어서/Adjective': 8094,
 '당황/Noun': 2374,
 '했고/Verb': 10541,
 '너무/Adverb': 1806,
 '밍밍/Noun': 4331,
 '했어요/Verb': 10569,
 '와/Verb': 7368,
 '여길/Verb': 7089,
 '왜/Noun': 7426,
 '이제/Noun': 7843,
 '알았죠/Verb': 6701,
 '맛있고/Adjective': 3613,
 '반찬/Noun': 4449,
 '하나/Noun': 10150,
 '다/Adverb': 2141,
 '맛있네요/Adjective': 3623,
 '자주/Noun': 8172,
 '갈게요/Verb': 313,
 '오늘/Noun': 7230,
 '의/Josa': 7677,
 '메뉴/Noun': 3996,
 '좋아요/Adjective': 8605,
 '어쩜/Noun': 6892,
 '갖가지/Noun': 430,
 '요리/Noun': 7462,
 '를/Josa': 3267,
 '그렇게/Adverb': 1057,
 '잘/VerbPrefix': 8197,
 '하시죠/Verb': 10262,
 '이런/A

In [23]:
# 첫번째 행의 리뷰 이름
print(df["comment"][100])
# 첫번째 행의 긍정/부정 시그널
print(df["target"][100])
# 첫번째 행의 index_vectorizer의 (형태소, 시리얼넘버)  
print(X[100])

이전하고 나서 대기도 많이 줄고 훨씬 쾌적해졌어요 여전히 너무 맛있고 하 야채배카레 있었으면 근데 옮긴 곳 인테리어가 좀 희한해요 미용실혹은 히비스커스 아이스티 팔것같은 카페그런 느낌
1
  (0, 8781)	1
  (0, 714)	1
  (0, 1806)	1
  (0, 3613)	1
  (0, 77)	1
  (0, 2529)	1
  (0, 9520)	1
  (0, 10118)	1
  (0, 8549)	1
  (0, 799)	1
  (0, 2384)	1
  (0, 3517)	1
  (0, 4540)	1
  (0, 7928)	1
  (0, 2087)	1
  (0, 1094)	1
  (0, 7841)	1
  (0, 1562)	1
  (0, 10771)	1
  (0, 9601)	1
  (0, 7116)	1
  (0, 10105)	1
  (0, 6757)	1
  (0, 8112)	1
  (0, 7361)	1
  (0, 10807)	1
  (0, 4289)	1
  (0, 10663)	1
  (0, 10816)	1
  (0, 6443)	1
  (0, 9896)	1
  (0, 473)	1
  (0, 9532)	1
  (0, 1040)	1


## TF IDF 벡터생성


In [24]:
tfidf_vectorizer = TfidfTransformer()

X = tfidf_vectorizer.fit_transform(X)
print(X[0])

  (0, 10493)	0.7660824489392268
  (0, 7239)	0.6427423133941603


In [25]:
X[0].shape

(1, 10844)

In [26]:
X.shape

(2522, 10844)

## 학습/테스트 데이터 분리

In [27]:
y = df['target']

x_train, x_test , y_train, y_test  = train_test_split(X, y,test_size=0.2, stratify=y, random_state=22)

## KNN, LogisticRegression 모델
---
* Grid Search로 최적모델 찾기

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
from sklearn.model_selection import KFold


params = {
	"n_neighbors" : list(range(1, 11))
}

model1 = KNeighborsClassifier()

kfold = KFold(n_splits=10, shuffle=True, random_state = 1)
grid_clf1 = GridSearchCV(model1, param_grid = params, scoring="f1", cv=kfold)
grid_clf1.fit(x_train, y_train)

# 결과 출력
print(f"최적의 하이퍼 파라미터 : {grid_clf1.best_params_}")
print(f"최적의 모델 평균 성능 : {grid_clf1.best_score_}")

최적의 하이퍼 파라미터 : {'n_neighbors': 10}
최적의 모델 평균 성능 : 0.81909638456577


In [30]:
params = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

model2 = LogisticRegression()

grid_clf2 = GridSearchCV(model2, param_grid=params, scoring='f1', cv=kfold)
grid_clf2.fit(x_train, y_train)

print(f'최적의 하이퍼 파라미터 : {grid_clf2.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf2.best_score_}')

최적의 하이퍼 파라미터 : {'C': 10, 'penalty': 'l2'}
최적의 모델 평균 성능 : 0.862956708269609


In [31]:
# 모델 평가"
model = grid_clf2.best_estimator_
model.score(x_test, y_test)

0.805940594059406

## 감정분석

In [32]:
def sentiment_predict(new_sentence):
    v_data = index_vectorizer.transform(new_sentence)
    X = tfidf_vectorizer.transform(v_data)
    y = model.predict(X)

    if y == 0:
        print(f'{new_sentence[0]} -> 부정')
    else :
        print(f'{new_sentence[0]} -> 긍정')

In [33]:
sentiment_predict(['음식도 맛없고 직원도 불친절해요'])
sentiment_predict(['주방장님이 엄청 친절해요'])
sentiment_predict(['서비스나 맛이 다 별로에요'])
sentiment_predict(['우리 엄마가 해준 것보다 나은듯'])
sentiment_predict(['여기 좀 심각합니다'])
sentiment_predict(['양 실화냐'])
sentiment_predict(['직원이 미친듯'])
sentiment_predict(['하.. 그냥 할말이 없음 전체적으로 왜 그 모양인지 모르겠음'])
sentiment_predict(['와 대박 부모님 모시고 싶은 맛'])
sentiment_predict(['담에 또 와야지'])
sentiment_predict(['나쁘지 않았음 다음 번에 가면 다른 메뉴 먹을 예정'])
sentiment_predict(['진짜 쓰레기'])
sentiment_predict(['개쓰레기 같은 맛 걍 장사 접어라'])

음식도 맛없고 직원도 불친절해요 -> 부정
주방장님이 엄청 친절해요 -> 긍정
서비스나 맛이 다 별로에요 -> 부정
우리 엄마가 해준 것보다 나은듯 -> 긍정
여기 좀 심각합니다 -> 긍정
양 실화냐 -> 긍정
직원이 미친듯 -> 부정
하.. 그냥 할말이 없음 전체적으로 왜 그 모양인지 모르겠음 -> 부정
와 대박 부모님 모시고 싶은 맛 -> 긍정
담에 또 와야지 -> 긍정
나쁘지 않았음 다음 번에 가면 다른 메뉴 먹을 예정 -> 긍정
진짜 쓰레기 -> 긍정
개쓰레기 같은 맛 걍 장사 접어라 -> 부정
